Generating Data

In [1]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 16

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    './Dataset/train_set/',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    './Dataset/test_set/',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary'
)

Found 436 images belonging to 2 classes.
Found 121 images belonging to 2 classes.


Building Model

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['acc']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

Training and saving model

In [3]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)
model.save('forest1.h5')

Epoch 1/10
28/28 [==============================] - 26s 889ms/step - loss: 0.5889 - acc: 0.7317 - val_loss: 0.1624 - val_acc: 0.9669
Epoch 2/10
28/28 [==============================] - 15s 529ms/step - loss: 0.3868 - acc: 0.8417 - val_loss: 0.0747 - val_acc: 0.9669
Epoch 3/10
28/28 [==============================] - 15s 530ms/step - loss: 0.3402 - acc: 0.8716 - val_loss: 0.0937 - val_acc: 0.9669
Epoch 4/10
28/28 [==============================] - 15s 547ms/step - loss: 0.2925 - acc: 0.8899 - val_loss: 0.0580 - val_acc: 0.9835
Epoch 5/10
28/28 [==============================] - 15s 540ms/step - loss: 0.3368 - acc: 0.8853 - val_loss: 0.0639 - val_acc: 0.9835
Epoch 6/10
28/28 [==============================] - 15s 527ms/step - loss: 0.2273 - acc: 0.9197 - val_loss: 1.1932 - val_acc: 0.7934
Epoch 7/10
28/28 [==============================] - 14s 519ms/step - loss: 0.2504 - acc: 0.9060 - val_loss: 0.1352 - val_acc: 0.9504
Epoch 8/10
28/28 [==============================] - 15s 528ms/step - 

Fire Detection

In [4]:
import cv2
import numpy as np
from keras.utils import load_img, img_to_array
from keras.models import load_model
from twilio.rest import Client
from playsound import playsound

model = load_model(r'forest1.h5')
video = cv2.VideoCapture(0)
name = ['forest', 'with fire']

while(1):
    success, frame = video.read()
    cv2.imwrite("image.jpg", frame)
    img = load_img("image.jpg", target_size = (150,150))
    x = img_to_array(img)
    x = np.expand_dims(x,axis = 0)
    pred = model.predict(x)
    classes_x=np.argmax(pred,axis=1)
    p = pred[0][0].astype(int)
    print(pred)
    
    cv2.putText(frame, "predicted class ="+str(name[p]), (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0),1)
    
    pred = model.predict(x)
    if pred[0]==1:
        account_sid = 'AC03faa7b579ff4212e3a310a93e553e1a'
        auth_token = 'c6e2eaada8fdabad91c463189525cf37'
        client = Client(account_sid, auth_token)
        message = client.messages \
            .create(
                body = 'Forest Fire is detected, stay alert',
                from_='+12182559719',
                to='+919600909256'
            )
        print(message.sid)
        print('Fire Detected')
        print('SMS sent!')
        playsound(r'C:/Users/SMTEC/Downloads/alarm.mp3')
    else:
        print("No Danger")
    cv2.imshow("image", frame)
    
    if cv2.waitKey(1) & 0xFF==ord('a'):
        break
video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 114ms/step
[[0.]]
1/1 [==============================] - 0s 26ms/step
No Danger
1/1 [==============================] - 0s 27ms/step
[[0.]]
1/1 [==============================] - 0s 30ms/step
No Danger
1/1 [==============================] - 0s 29ms/step
[[0.]]
1/1 [==============================] - 0s 32ms/step
No Danger
1/1 [==============================] - 0s 28ms/step
[[0.]]
1/1 [==============================] - 0s 32ms/step
No Danger
1/1 [==============================] - 0s 30ms/step
[[0.]]
1/1 [==============================] - 0s 37ms/step
No Danger
1/1 [==============================] - 0s 29ms/step
[[0.]]
1/1 [==============================] - 0s 28ms/step
No Danger
1/1 [==============================] - 0s 30ms/step
[[0.]]
1/1 [==============================] - 0s 34ms/step
No Danger
1/1 [==============================] - 0s 33ms/step
[[0.]]
1/1 [==============================] - 0s 37ms/step
No Danger
1/1 [==========================